In [1]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import re

In [26]:
def kmib_crawling(start_date, end_date):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    eng_url_head = 'http://www.kmib.co.kr/search/searchResult.asp?searchWord=kaist'
    kor_url_head = 'http://www.kmib.co.kr/search/searchResult.asp?searchWord=%uCE74%uC774%uC2A4%uD2B8'
    url_tail = '&startDate='+str(start_date)+'&endDate='+ str(end_date) + "&pageNo="
    
    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 1
    while True:
        url_final = eng_url_head + url_tail + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.search_nws')) == 0:
                break
            elements = soup.select('.search_nws a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 1
    while True:
        url_final = kor_url_head + url_tail + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.search_nws')) == 0:
                break
            elements = soup.select('.search_nws a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_2')
        for element in elements:
            link = element.attrs['href']
            url2.append(link)
        page += 1
    url = list(set(url1).union(set(url2)))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link =  getattr(row, 'url')
        response = requests.get(link, headers=headers)
        html = response.content.decode('euc-kr','replace')
        soup = bs(html,'html.parser')
        try:
            for script in soup(["script", "style"]):
                script.decompose() 
            title = soup.select('title')[0].get_text()
            content = soup.select('#article')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            contents.append(content)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/kmib.xlsx",index = False)
    df.to_pickle("pickle files/kmib.pkl")

In [27]:
kmib_crawling('2015-01-01', '2020-10-23')

336it [00:24, 10.91it/s]

Pandas(Index=332, url='http://news.kmib.co.kr/article/view.asp?arcid=0012001510')


1144it [01:28, 12.90it/s]
